<a href="https://colab.research.google.com/github/mortgad/DLVR/blob/main/IMDB_WIKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tarfile
import urllib.request

# Function to download and extract tar files
def download_and_extract(url, destination_path):
    # File name based on URL
    file_name = url.split('/')[-1]
    file_path = os.path.join(destination_path, file_name)
    
    # Download the file
    print(f"Downloading {file_name}...")
    urllib.request.urlretrieve(url, file_path)
    print(f"Downloaded {file_name}.")

    # Extract the tar file
    print(f"Extracting {file_name}...")
    with tarfile.open(file_path, "r") as tar:
        tar.extractall(destination_path)
    print(f"Extracted {file_name}.")

    # Remove the tar file after extraction to save space
    os.remove(file_path)
    print(f"Removed {file_name}.")

# Define URLs and destination path
urls = [
    "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar",
    "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar"
]
destination_path = "/content/datasets"

# Create the destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# Download and extract each dataset
for url in urls:
    download_and_extract(url, destination_path)

print("All datasets downloaded and extracted.")